# Viewing angles

Visualizing the illumination and viewing angles. Explaining how ViSiProg groups are related?

In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

import plotly.graph_objs as go
from visiprog.data import read_VSP_label, read_img_list, get_viewing_condition

In [2]:
def polar_to_euclidean(theta, phi):
    '''
    theta is polar angle (angle from z axis)
    phi is azimuth angle (angle from x axis to projection)
    
    '''
    X = np.sin(theta) * np.cos(phi)
    Y = np.sin(theta) * np.sin(phi)
    Z = np.cos(theta)
    return (X, Y, Z)

In [3]:
viewing = np.genfromtxt('visiprog/data/viewing.csv',delimiter=',')
print(viewing.shape)

(205, 5)


In [4]:
trace_illum_x = []
trace_illum_y = []
trace_illum_z = []

trace_viewing_x = []
trace_viewing_y = []
trace_viewing_z = []

for i in range(viewing.shape[0]):
    illum_x, illum_y, illum_z = polar_to_euclidean(viewing[i,3],viewing[i,4])
    view_x, view_y, view_z = polar_to_euclidean(viewing[i,1],viewing[i,2])
    
    line_x = [illum_x, 0, view_x]
    line_y = [illum_y, 0, view_y]
    line_z = [illum_z, 0, view_z]
    
    trace_illum_x.append(illum_x)
    trace_illum_y.append(illum_y)
    trace_illum_z.append(illum_z)
    
    trace_viewing_x.append(view_x)
    trace_viewing_y.append(view_y)
    trace_viewing_z.append(view_z)

    
illum_scatter = go.Scatter3d(
    x=trace_illum_x,
    y=trace_illum_y,
    z=trace_illum_z,
    mode='markers',
    marker=dict(
        size=12,
        line=dict(
            color='rgba(217, 217, 217, 0.14)',
            width=0.5
        ),
        opacity=0.8
    )
)

In [6]:
layout = go.Layout(
    title='Illumination angles',
    autosize=True,
    width=900,
    height=800,
    margin=dict(
        l=65,
        r=50,
        b=65,
        t=90
    )
)
fig = go.Figure(data=[illum_scatter], layout=layout)
iplot(fig, filename='illumination')

In [7]:
groups, N = read_VSP_label()

list_img = read_img_list()

traces = []

for i in groups[100]: 
    viewing_index = get_viewing_condition(list_img[i])
    
    illum_x, illum_y, illum_z = polar_to_euclidean(viewing[viewing_index,3],viewing[viewing_index,4])
    view_x, view_y, view_z = polar_to_euclidean(viewing[viewing_index,1],viewing[viewing_index,2])
    
    line_x = [illum_x, 0, view_x]
    line_y = [illum_y, 0, view_y]
    line_z = [illum_z, 0, view_z]
    
    # plotting line
    line_scatter = go.Scatter3d(
        x=line_x,
        y=line_y,
        z=line_z,
        name="pair_{}".format(i)
    )
    
    traces.append(line_scatter)
    

    
illum_scatter = go.Scatter3d(
    x=trace_illum_x,
    y=trace_illum_y,
    z=trace_illum_z,
    mode='markers',
    marker=dict(
        size=12,
        line=dict(
            color='rgba(217, 217, 217, 0.14)',
            width=0.5
        ),
        opacity=0.8
    )
)

Plotting the illumination x,y,z

In [8]:
layout = go.Layout(
    title='Illumination angles',
    autosize=True,
    width=900,
    height=800,
    margin=dict(
        l=65,
        r=50,
        b=65,
        t=90
    )
)
fig = go.Figure(data=traces, layout=layout)
iplot(fig, filename='elevations-3d-surface')